In [1]:
%matplotlib widget

import os
import sys

sys.path.insert(0, os.getenv("HOME") + "/pycode/MscThesis/")
import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_plate_number,
    get_postion_number,
    get_begin_index,
)
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import (
    Experiment,
    Edge,
    Node,
    Hyphae,
    plot_raw_plus,
)
from amftrack.pipeline.paths.directory import (
    run_parallel,
    find_state,
    directory_scratch,
    directory_project,
)
from amftrack.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests
from amftrack.pipeline.functions.hyphae_id_surf import get_pixel_growth_and_new_children
from collections import Counter
from IPython.display import clear_output
from amftrack.notebooks.analysis.data_info import *
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
plt.rcParams.update(
    {"font.family": "verdana", "font.weight": "normal", "font.size": 20}
)
from amftrack.plotutil import plot_node_skel

In [2]:
lapse = 60
exp = get_exp((38, 131, 131 + lapse), directory_project)

begin = 2021-08-18 13:24:00 
  end = 2021-08-23 15:24:00


In [3]:
exp2 = Experiment(38, directory_project)
exp2.copy(exp)

In [4]:
exp = exp2

In [5]:
def transform_skeleton_final_for_show(skeleton_doc, Rot, trans):
    skeleton_transformed = {}
    transformed_keys = np.round(
        np.transpose(np.dot(Rot, np.transpose(np.array(list(skeleton_doc.keys())))))
        + trans
    ).astype(np.int)
    i = 0
    for pixel in list(transformed_keys):
        i += 1
        skeleton_transformed[(pixel[0], pixel[1])] = 1
    skeleton_transformed_sparse = sparse.lil_matrix((27000, 60000))
    for pixel in list(skeleton_transformed.keys()):
        i += 1
        skeleton_transformed_sparse[(pixel[0], pixel[1])] = 1
    return skeleton_transformed_sparse


def get_skeleton_non_aligned(exp, boundaries, t, directory):
    i = t
    plate = exp.plate
    listdir = os.listdir(directory)
    dates = exp.dates
    date = dates[i]
    directory_name = get_dirname(date, plate)
    path_snap = directory + directory_name
    skel = read_mat(path_snap + "/Analysis/skeleton.mat")
    skelet = skel["skeleton"]
    skelet = sparse_to_doc(skelet)
    #     Rot= skel['R']
    #     trans = skel['t']
    skel_aligned = transform_skeleton_final_for_show(
        skelet, np.array([[1, 0], [0, 1]]), np.array([0, 0])
    )
    output = skel_aligned[
        boundaries[2] : boundaries[3], boundaries[0] : boundaries[1]
    ].todense()
    kernel = np.ones((5, 5), np.uint8)
    output = cv2.dilate(output.astype(np.uint8), kernel, iterations=1)
    return output

In [12]:
from amftrack.util.sys import get_skeleton


def plot_raw_plus_random(exp, compress=5, ranges=1000):
    t0 = choice(range(exp.ts))
    node_ch = choice(
        [node for node in exp.nodes if node.is_in(t0) and node.degree(t0) == 1]
    )
    #     node_ch = choice(exp.nodes)
    #     t0 = choice(node_ch.ts())
    node_ch.show_source_image(t0, t0 + 1)
    for index, t in enumerate([t0, t0 + 1]):
        date = exp.dates[t]
        anchor_time = t0
        center = node_ch.pos(anchor_time)[1], node_ch.pos(anchor_time)[0]
        window = (
            center[0] - ranges,
            center[0] + ranges,
            center[1] - ranges,
            center[1] + ranges,
        )
        skelet = get_skeleton_non_aligned(exp, window, t, exp.directory)
        tips = [
            node.label
            for node in exp.nodes
            if t in node.ts()
            and node.degree(t) == 1
            and node.pos(t)[1] >= window[0] - ranges
            and node.pos(t)[1] <= window[1] + ranges
            and node.pos(t)[0] >= window[2] - ranges
            and node.pos(t)[0] <= window[3] + ranges
        ]
        junction = [
            node.label
            for node in exp.nodes
            if t in node.ts()
            and node.degree(t) >= 2
            and node.pos(t)[1] >= window[0] - ranges
            and node.pos(t)[1] <= window[1] + ranges
            and node.pos(t)[0] >= window[2] - ranges
            and node.pos(t)[0] <= window[3] + ranges
        ]
        directory_name = get_dirname(date, exp.plate)
        path_snap = exp.directory + directory_name
        skel = read_mat(path_snap + "/Analysis/skeleton_pruned_realigned.mat")
        Rot = skel["R"]
        trans = skel["t"]
        im = read_mat(path_snap + "/Analysis/raw_image.mat")["raw"]
        size = 8
        fig = plt.figure(figsize=(9, 9))
        ax = fig.add_subplot(111)
        ax.imshow(
            im[
                (window[2] // compress) : (window[3] // compress),
                (window[0] // compress) : (window[1] // compress),
            ]
        )
        ax.imshow(
            cv2.resize(skelet, (2 * ranges // compress, 2 * ranges // compress)),
            alpha=0.2,
        )
        shift = (window[2], window[0])
        greys = [1, 0.5]
        for i, node_list in enumerate([tips, junction]):
            grey = greys[i]
            bbox = dict(boxstyle="circle", fc=colors.rgb2hex((grey, grey, grey)))
            #             ax.text(right, top, time,
            #                 horizontalalignment='right',
            #                 verticalalignment='bottom',
            #                 transform=ax.transAxes,color='white')
            for node in node_list:
                #                     print(self.positions[ts[i]])
                if node in exp.positions[t].keys():
                    xs, ys = exp.positions[t][node]
                    rottrans = np.dot(np.linalg.inv(Rot), np.array([xs, ys] - trans))
                    ys, xs = round(rottrans[0]), round(rottrans[1])
                    tex = ax.text(
                        (xs - shift[1]) // compress,
                        (ys - shift[0]) // compress,
                        str(node),
                        ha="center",
                        va="center",
                        size=size,
                        bbox=bbox,
                    )
        plt.show()

In [15]:
plt.close("all")
plot_raw_plus_random(exp, compress=5, ranges=700)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …